# Carga de audios, transcripción, limpieza y almacenamiento

In [4]:
import os
#import mysql.connector
import re

# Install whisper package
#%pip install openai-whisper

import whisper
import subprocess

In [5]:
# Folderes donde se ubican los audios y donde se almacenan los txt
audio_folder = "C:/Users/Usuario/tfm_call_optimizer/audios/"
transcription_folder = "C:/Users/Usuario/tfm_call_optimizer/transcripciones"

# Connect to MySQL
conn = mysql.connector.connect(
    host="your-mysql-server",
    user="your-username",
    password="your-password",
    database="your-database"
)
cursor = conn.cursor()

In [6]:
# Asegurémonos de que `ffmpeg` esté en el PATH antes de la transcripción
ffmpeg_path = r"C:\Users\Usuario\ffmpeg-2025-03-06-git-696ea1c223-essentials_build\bin\ffmpeg.exe"

# Verificar si ffmpeg está accesible desde Python
try:
    subprocess.run([ffmpeg_path, "-version"], check=True, capture_output=True)
    print("✅ ffmpeg está correctamente accesible.")
except FileNotFoundError:
    print("❌ No se pudo encontrar ffmpeg en la ruta especificada.")
    exit()


✅ ffmpeg está correctamente accesible.


In [7]:
filename = "3_nodasolucion_vivi.wav"  # Nombre del archivo de prueba con extensión
audio_path = os.path.join(audio_folder, filename)
transcription_path = os.path.join(transcription_folder, filename.replace(".wav", ".txt"))

print(f"Buscando archivo en: {audio_path}")
if os.path.exists(audio_path):
    print("✅ El archivo existe.")
else:
    print("❌ El archivo NO existe. Revisa la ruta.")

# Transcribir audio
#result = model.transcribe(audio_path,shell=True)
#raw_text = result["text"]


Buscando archivo en: C:/Users/Usuario/tfm_call_optimizer/audios/3_nodasolucion_vivi.wav
✅ El archivo existe.


In [8]:
# Verificar si ffmpeg está accesible desde Python
try:
    subprocess.run([ffmpeg_path, "-version"], check=True, capture_output=True)
    print("✅ ffmpeg está correctamente accesible.")
except FileNotFoundError:
    print("❌ No se pudo encontrar ffmpeg en la ruta especificada.")
    exit()

✅ ffmpeg está correctamente accesible.


In [9]:
# Asegurémonos de que `ffmpeg` esté en el PATH antes de la transcripción
ffmpeg_path = r"C:\Users\Usuario\ffmpeg-2025-03-06-git-696ea1c223-essentials_build\bin\ffmpeg.exe"

In [10]:
# Configurar ffmpeg manualmente en whisper
whisper.utils.ffmpeg_bin = ffmpeg_path  # Esto le dice a whisper dónde encontrar ffmpeg

In [11]:
# Load Whisper model
model = whisper.load_model("medium")

c:\Users\Usuario\anaconda3\envs\chatbot_rag\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location

# Ruta del archivo WAV
input_audio_path = "C:/Users/Usuario/tfm_call_optimizer/audios/3_nodasolucion_vivi.WAV"
output_audio_path = "C:/Users/Usuario/tfm_call_optimizer/audios/3_nodasolucion_vivi_converted.mp3"

# Comando para convertir WAV a MP3
ffmpeg_command = [
    r"C:\Users\Usuario\ffmpeg-2025-03-06-git-696ea1c223-essentials_build\bin\ffmpeg.exe",
    "-i", input_audio_path,
    output_audio_path
]

# Ejecutar el comando
try:
    subprocess.run(ffmpeg_command, check=True)
    print(f"✅ El archivo se ha convertido exitosamente a: {output_audio_path}")
except subprocess.CalledProcessError as e:
    print(f"❌ Error en la conversión: {e}")

# Ahora, usa este archivo convertido para transcribirlo con whisper
audio_path = output_audio_path

In [12]:
# Transcribir el archivo de audio convertido
audio_path = "C:/Users/Usuario/tfm_call_optimizer/audios/3_nodasolucion_vivi_converted.mp3"

# Transcribir audio
result = model.transcribe(audio_path)

# Obtener el texto transcrito
raw_text = result["text"]

# Imprimir el texto transcrito
print(f"Texto transcrito: {raw_text}")

# Guardar la transcripción en un archivo de texto
transcription_path = "C:/Users/Usuario/tfm_call_optimizer/transcripciones/3_nodasolucion_vivi_converted.txt"
with open(transcription_path, "w", encoding="utf-8") as f:
    f.write(raw_text)

print(f"✅ Transcripción guardada en: {transcription_path}")

c:\Users\Usuario\anaconda3\envs\chatbot_rag\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Texto transcrito:  El número de la extensión con la cual desea comunicarte, por favor, márquelo. De lo contrario, marque. 1. Pedidos de operación. 2. Despacio. 3. Cotizaciones. Si desea comunicarle con nuestra operadora, por favor, marque cero. Dios, dices Buenas tardes. ¿Le habla Lina? Hola Lina, ¿cómo te vas a hablar con yo? Mi nombre es Satian. Hola Johnny, muy bien, gracias por tu malestar. Ah, muy bien, gracias. Johnny, es que me ha pasado algo extraño con el computador. Todas las líneas no me funcionaban, ninguna de las computadores, salía a marcar a cualquier extensión y no se dejaba, era en verde. Y en el día en que el teléfono se puso todo como en rojo. Entonces me tocó desconectarlo del internet y desconectarlo de la luz y volver a conectar. Y volvió, reinició y reinició. Ya viene, así me salen y me entran las llamadas. En ese momento acaba de ser como que lo reinicié y hacía todo, pero quería que lo hiciera como monitoreo porque si me parece extraño lo que pensó. Ah, no, dal

In [ ]:
# Limpiar transcripción
cleaned_text = clean_transcription(raw_text)

# Guardar en archivo de texto
with open(transcription_path, "w", encoding="utf-8") as f:
    f.write(cleaned_text)

print(f"✅ Saved cleaned transcription: {transcription_path}")


# Function to clean transcription text
def clean_transcription(text):
    """
    Cleans the transcription text by removing filler words, extra spaces,
    and unwanted artifacts.
    """
    # Define common filler words (can be extended)
    filler_words = ["eh", "mmm", "este", "o sea", "pues", "bueno", "entonces", "verdad", "sabes", "digo"]
    filler_pattern = r'\b(?:' + '|'.join(filler_words) + r')\b'

    # Remove filler words
    text = re.sub(filler_pattern, '', text, flags=re.IGNORECASE)

    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)

    # Remove unnecessary punctuation repetition (e.g., "Hello...." → "Hello.")
    text = re.sub(r'([.?!])\1+', r'\1', text)

    # Strip leading/trailing spaces
    text = text.strip()

    return text



for filename in os.listdir(audio_folder):
    if filename.endswith(".wav") or filename.endswith(".mp3"):
        audio_path = os.path.join(audio_folder, filename)
        transcription_path = os.path.join(transcription_folder, filename.replace(".wav", ".txt").replace(".mp3", ".txt"))

        # Transcribe audio
        result = model.transcribe(audio_path)
        raw_text = result["text"]

        # Clean transcription text
        cleaned_text = clean_transcription(raw_text)

        # Save cleaned transcription to a .txt file
        with open(transcription_path, "w", encoding="utf-8") as f:
            f.write(cleaned_text)

        # Insert cleaned transcription into MySQL
        sql = """
        INSERT INTO transcriptions (call_id, transcription, status)
        VALUES (%s, %s, %s)
        """
        values = (filename.replace(".wav", "").replace(".mp3", ""), cleaned_text, "Pending")

        cursor.execute(sql, values)
        conn.commit()

        print(f"✅ Saved cleaned transcription: {transcription_path} & stored in database.")

cursor.close()
conn.close()
